In [ ]:
# !pip install 'sagemaker[local]' sagemaker --upgrade 

In [19]:
!cd ../ && ./build_and_push.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  44.76MB
Step 1/11 : FROM python:3.7.6-slim-buster
 ---> 84de2ffd919d
Step 2/11 : COPY requirements.txt /opt/program/requirements.txt
 ---> Using cache
 ---> 5e218ddefc50
Step 3/11 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          nginx          ca-certificates          git          python3-dev          gcc     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 6a3b064a2c32
Step 4/11 : RUN pip install --no-cache-dir -r /opt/program/requirements.txt
 ---> Using cache
 ---> 663fbc8ce76a
Step 5/11 : RUN pip install --upgrade mxnet     && pip install autogluon
 ---> Using cache
 ---> 791595f2b584

In [20]:
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_serializer


sess = sagemaker.Session()
role = get_execution_role()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [21]:
# training_data = 's3://autogluon/datasets/Inc/train.csv'
# test_data = 's3://autogluon/datasets/Inc/test.csv'
training_data = 's3://vpon-workshop-yianc/train.csv'
test_data = 's3://vpon-workshop-yianc/test.csv'

In [18]:
!aws s3 cp s3://autogluon/datasets/Inc/train.csv . 
!cat train.csv 

download: s3://autogluon/datasets/Inc/train.csv to ./train.csv    
age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
25, Private,178478, Bachelors,13, Never-married, Tech-support, Own-child, White, Female,0,0,40, United-States, <=50K
23, State-gov,61743, 5th-6th,3, Never-married, Transport-moving, Not-in-family, White, Male,0,0,35, United-States, <=50K
46, Private,376789, HS-grad,9, Never-married, Other-service, Not-in-family, White, Male,0,0,15, United-States, <=50K
55, ?,200235, HS-grad,9, Married-civ-spouse, ?, Husband, White, Male,0,0,50, United-States, >50K
36, Private,224541, 7th-8th,4, Married-civ-spouse, Handlers-cleaners, Husband, White, Male,0,0,40, El-Salvador, <=50K
51, Private,178054, Some-college,10, Married-civ-spouse, Sales, Husband, White, Male,0,0,40, ?, >50K
33, Private,263561, Some-college,10, Married-civ-spouse, Craft-repair, Husband, White, Male,0,0,60, United-

38, Private,247547, HS-grad,9, Never-married, Other-service, Unmarried, Black, Female,0,0,40, United-States, <=50K
47, Federal-gov,169549, Bachelors,13, Married-civ-spouse, Prof-specialty, Husband, White, Male,7298,0,60, ?, >50K
48, Private,235891, HS-grad,9, Never-married, Other-service, Not-in-family, White, Male,0,0,40, Puerto-Rico, <=50K
66, Private,207917, 7th-8th,4, Married-civ-spouse, Other-service, Husband, Black, Male,1797,0,20, United-States, <=50K
37, Private,359001, Assoc-voc,11, Married-civ-spouse, Transport-moving, Husband, White, Male,15024,0,50, United-States, >50K
49, Private,166789, HS-grad,9, Married-civ-spouse, Adm-clerical, Husband, White, Male,0,0,40, United-States, <=50K
39, Private,179481, Bachelors,13, Married-civ-spouse, Sales, Husband, White, Male,0,0,40, United-States, <=50K
43, Private,216685, HS-grad,9, Married-civ-spouse, Prof-specialty, Wife, White, Female,0,0,35, United-States, <=50K
46, Local-gov,197042, Some-college,10, Married-civ-spouse, Exec-manage

22, Private,317019, Some-college,10, Never-married, Other-service, Not-in-family, White, Female,0,0,30, United-States, <=50K
42, Private,252518, Bachelors,13, Married-civ-spouse, Exec-managerial, Husband, White, Male,0,0,70, ?, <=50K
30, Private,201122, Bachelors,13, Married-civ-spouse, Prof-specialty, Husband, White, Male,0,0,40, United-States, >50K
28, Private,173649, HS-grad,9, Never-married, Other-service, Own-child, Black, Female,0,0,40, ?, <=50K
23, Private,161708, Some-college,10, Never-married, Other-service, Unmarried, White, Female,0,0,20, United-States, <=50K
44, Self-emp-not-inc,124692, HS-grad,9, Married-civ-spouse, Prof-specialty, Husband, White, Male,0,1902,40, United-States, >50K
38, Private,374983, Bachelors,13, Married-civ-spouse, Prof-specialty, Husband, White, Male,0,0,50, United-States, <=50K
33, Private,336061, HS-grad,9, Married-civ-spouse, Exec-managerial, Husband, White, Male,0,0,50, United-States, <=50K
35, Private,308691, Masters,14, Married-civ-spouse, Machi

30, Self-emp-inc,224498, HS-grad,9, Divorced, Adm-clerical, Unmarried, White, Female,0,0,40, United-States, <=50K
21, Private,124589, HS-grad,9, Never-married, Sales, Own-child, White, Female,0,0,40, United-States, <=50K
63, Local-gov,199275, HS-grad,9, Never-married, Exec-managerial, Not-in-family, White, Female,8614,0,38, United-States, >50K
23, Private,133239, Assoc-voc,11, Never-married, Tech-support, Not-in-family, White, Female,0,0,40, United-States, <=50K
73, Local-gov,143437, HS-grad,9, Married-civ-spouse, Exec-managerial, Husband, White, Male,0,0,20, United-States, <=50K
39, Private,347434, 9th,5, Married-civ-spouse, Handlers-cleaners, Husband, White, Male,0,0,43, Mexico, <=50K
32, Private,195891, Bachelors,13, Never-married, Adm-clerical, Not-in-family, White, Male,0,0,55, United-States, <=50K
39, Private,116546, Some-college,10, Married-civ-spouse, Sales, Husband, White, Male,0,0,50, United-States, >50K
72, Self-emp-inc,172407, Prof-school,15, Married-civ-spouse, Prof-specia

28, Private,55191, Assoc-acdm,12, Never-married, Handlers-cleaners, Not-in-family, White, Male,0,0,45, United-States, <=50K
41, ?,213416, 5th-6th,3, Married-civ-spouse, ?, Husband, White, Male,0,0,32, Mexico, <=50K
62, Private,195543, Bachelors,13, Married-civ-spouse, Sales, Husband, White, Male,0,0,20, United-States, >50K
38, Local-gov,73715, Some-college,10, Married-civ-spouse, Protective-serv, Husband, White, Male,0,0,60, United-States, >50K
32, Private,182323, 9th,5, Married-civ-spouse, Craft-repair, Husband, White, Male,0,0,40, United-States, <=50K
30, Private,29662, Assoc-acdm,12, Married-civ-spouse, Other-service, Wife, White, Female,0,0,25, United-States, >50K
56, Local-gov,264436, Some-college,10, Divorced, Exec-managerial, Not-in-family, White, Female,0,0,40, United-States, <=50K
41, Self-emp-not-inc,34037, HS-grad,9, Married-civ-spouse, Farming-fishing, Husband, White, Male,0,0,99, United-States, <=50K
33, Private,112900, HS-grad,9, Divorced, Other-service, Unmarried, White,

42, Private,136986, HS-grad,9, Married-civ-spouse, Exec-managerial, Husband, White, Male,0,0,35, United-States, >50K
19, Private,140499, HS-grad,9, Never-married, Protective-serv, Other-relative, White, Male,0,0,40, United-States, <=50K
31, Private,234537, Bachelors,13, Married-civ-spouse, Prof-specialty, Husband, White, Male,7298,0,50, United-States, >50K
21, State-gov,140764, Some-college,10, Never-married, Adm-clerical, Own-child, White, Female,0,0,12, United-States, <=50K
45, Private,241350, HS-grad,9, Married-civ-spouse, Handlers-cleaners, Husband, White, Male,0,0,48, United-States, >50K
39, Private,176296, Assoc-acdm,12, Married-civ-spouse, Protective-serv, Husband, Black, Male,0,0,60, United-States, <=50K
36, Private,311255, HS-grad,9, Married-civ-spouse, Machine-op-inspct, Husband, Black, Male,0,0,40, Haiti, <=50K
32, Self-emp-inc,103078, Bachelors,13, Married-civ-spouse, Sales, Husband, White, Male,0,0,40, United-States, <=50K
43, Private,256813, Masters,14, Married-civ-spouse

59, Private,108496, Masters,14, Married-civ-spouse, Exec-managerial, Husband, White, Male,0,0,45, United-States, >50K
27, Private,177057, HS-grad,9, Never-married, Machine-op-inspct, Not-in-family, White, Male,0,0,40, United-States, <=50K
41, Private,182217, Some-college,10, Divorced, Tech-support, Unmarried, White, Female,0,0,30, United-States, <=50K
54, ?,155755, HS-grad,9, Divorced, ?, Not-in-family, White, Female,4416,0,25, United-States, <=50K
29, Private,82242, Prof-school,15, Never-married, Prof-specialty, Unmarried, White, Male,27828,0,45, Germany, >50K
43, Local-gov,212206, Assoc-voc,11, Married-civ-spouse, Adm-clerical, Wife, White, Female,0,0,40, United-States, >50K
46, Federal-gov,330901, Some-college,10, Married-civ-spouse, Exec-managerial, Husband, Black, Male,0,0,40, United-States, <=50K
33, Private,50276, Bachelors,13, Never-married, Exec-managerial, Not-in-family, White, Male,0,0,40, United-States, <=50K
42, Local-gov,125461, Bachelors,13, Never-married, Exec-manageria

64, Private,166715, Some-college,10, Married-civ-spouse, Exec-managerial, Husband, White, Male,0,0,20, United-States, <=50K
63, Self-emp-not-inc,175177, Prof-school,15, Married-civ-spouse, Prof-specialty, Husband, White, Male,0,0,35, United-States, >50K
59, Private,64102, 10th,6, Married-civ-spouse, Handlers-cleaners, Husband, White, Male,0,0,40, United-States, <=50K
27, Private,128730, Assoc-voc,11, Never-married, Exec-managerial, Not-in-family, White, Male,10520,0,65, Greece, >50K
53, Federal-gov,218382, Assoc-acdm,12, Married-civ-spouse, Adm-clerical, Wife, Black, Female,0,0,40, United-States, >50K
26, Private,330695, Some-college,10, Married-civ-spouse, Machine-op-inspct, Husband, White, Male,0,0,40, United-States, <=50K
50, Self-emp-not-inc,158352, Masters,14, Divorced, Prof-specialty, Not-in-family, White, Female,0,0,25, United-States, <=50K
39, Private,257726, Some-college,10, Never-married, Machine-op-inspct, Own-child, White, Male,0,0,40, United-States, <=50K
72, Private,17776

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
image = 'autogluon-tabular-data'
image = '230755935769.dkr.ecr.us-east-1.amazonaws.com/autogluon-tabular-data'

sm_model = Estimator(image_name=image, 
                     role=role, 
                     train_instance_count=1, 
#                      train_instance_type='local',
                     train_instance_type='ml.c5.xlarge',
                     hyperparameters={'label-column': 'SalePrice'}
                    )

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [25]:
sm_model.fit({'training': training_data, 'testing': test_data})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-09-23 07:26:02 Starting - Starting the training job...
2020-09-23 07:26:04 Starting - Launching requested ML instances......
2020-09-23 07:27:34 Starting - Preparing the instances for training......
2020-09-23 07:28:19 Downloading - Downloading input data...
2020-09-23 07:28:45 Training - Downloading the training image.....Parameters: ['train.csv']
/opt/ml/input/data/training
        Id  MSSubClass MSZoning  ...  SaleType  SaleCondition SalePrice
0        1          60       RL  ...        WD         Normal    208500
1        2          20       RL  ...        WD         Normal    181500
2        3          60       RL  ...        WD         Normal    223500
3        4          70       RL  ...        WD        Abnorml    140000
4        5          60       RL  ...        WD         Normal    250000
...    ...         ...      ...  ...       ...            ...       ...
1455  1456          60       RL  ...        WD         Normal    175000
1456  1457          20       RL  ...    

In [26]:
predictor = sm_model.deploy(1, 'ml.m4.xlarge', serializer=json_serializer)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


---------------!

In [35]:
predictor

In [27]:
import pandas as pd
import json
import io

In [28]:
df = pd.read_csv('s3://vpon-workshop-yianc/test.csv')

In [59]:
record = df.head(2).T.to_dict()[1]
print(record) 
payload = json.dumps(record)
print(payload) 
predictor.content_type = 'application/json'
d = predictor.predict(payload)
d

# data = df.head()[['age', 'workclass', 'fnlwgt', 'education', 'education-num',
#        'marital-status', 'occupation', 'relationship', 'race', 'sex',
#        'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']]

{'Id': 1462, 'MSSubClass': 20, 'MSZoning': 'RL', 'LotFrontage': 81.0, 'LotArea': 14267, 'Street': 'Pave', 'Alley': nan, 'LotShape': 'IR1', 'LandContour': 'Lvl', 'Utilities': 'AllPub', 'LotConfig': 'Corner', 'LandSlope': 'Gtl', 'Neighborhood': 'NAmes', 'Condition1': 'Norm', 'Condition2': 'Norm', 'BldgType': '1Fam', 'HouseStyle': '1Story', 'OverallQual': 6, 'OverallCond': 6, 'YearBuilt': 1958, 'YearRemodAdd': 1958, 'RoofStyle': 'Hip', 'RoofMatl': 'CompShg', 'Exterior1st': 'Wd Sdng', 'Exterior2nd': 'Wd Sdng', 'MasVnrType': 'BrkFace', 'MasVnrArea': 108.0, 'ExterQual': 'TA', 'ExterCond': 'TA', 'Foundation': 'CBlock', 'BsmtQual': 'TA', 'BsmtCond': 'TA', 'BsmtExposure': 'No', 'BsmtFinType1': 'ALQ', 'BsmtFinSF1': 923.0, 'BsmtFinType2': 'Unf', 'BsmtFinSF2': 0.0, 'BsmtUnfSF': 406.0, 'TotalBsmtSF': 1329.0, 'Heating': 'GasA', 'HeatingQC': 'TA', 'CentralAir': 'Y', 'Electrical': 'SBrkr', '1stFlrSF': 1329, '2ndFlrSF': 0, 'LowQualFinSF': 0, 'GrLivArea': 1329, 'BsmtFullBath': 0.0, 'BsmtHalfBath': 0.0, 

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/autogluon-tabular-data-2020-09-23-07-26-02-284 in account 230755935769 for more information.

In [39]:
%%bash 
aws s3 cp s3://vpon-workshop-yianc/test.csv . 
payload=test.csv
content=${2:-text/csv}

curl --data-binary @${payload} -H "Content-Type: ${content}" -v https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/autogluon-tabular-data-2020-09-23-07-26-02-284/invocations


download: s3://vpon-workshop-yianc/test.csv to ./test.csv         
{"message":"Missing Authentication Token"}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 34.232.169.34:443...
* Connected to runtime.sagemaker.us-east-1.amazonaws.com (34.232.169.34) port 443 (#0)
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: /home/ec2-user/anaconda3/envs/python3/ssl/cacert.pem
  CApath: none
} [5 bytes data]
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
} [512 bytes data]
* TLSv1.3 (IN), TLS handshake, Server hello (2):
{ [91 bytes data]
* TLSv1.2 (IN), TLS handshake, Certificate (11):
{ [4928 bytes data]
* TLSv1.2 (IN), TLS handshake, Server key exchange (12):
{ [333 bytes data]
* TLSv1.2 (IN), TLS handshake, Server finished (14):
{ [4 bytes data]
* TLSv1.2 (OUT), TLS handshake, Client key exchange (16):
} [70 bytes data]
* TLSv1.2 (OUT), TLS change cipher, Change cip

In [62]:
predictor.content_type = 'text/csv'
payload = '1528,50,RL,60,8064,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Artery,Norm,1Fam,1.5Fin,6,8,1948,2004,Gable,CompShg,WdShing,Wd Shng,None,0,TA,TA,CBlock,TA,TA,No,ALQ,481,Rec,174,161,816,GasA,TA,Y,SBrkr,816,408,0,1224,1,0,1,0,3,1,TA,5,Typ,0,NA,Detchd,1950,Unf,1,280,TA,TA,Y,414,0,0,0,0,0,NA,GdWo,NA,0,5,2010,WD,Normal'
# f = open('test.csv', 'r')
# idx = 0 
# for l in f.readlines():
#     if idx > 0:
#         payload += l
#     idx += 1 
# print(payload)
d = predictor.predict(payload)
print(d)

b''


In [ ]:
predictor.delete_endpoint()